In [ ]:
from pathlib import Path

# ".." ile bir üst klasöre çıkıyoruz
BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"

PROC_DIR.mkdir(parents=True, exist_ok=True)

print("RAW_DIR exists:", RAW_DIR.exists(), "| Path:", RAW_DIR)


In [ ]:
from pathlib import Path
import pandas as pd

# 1) Doğru klasör yolu
BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"
PROC_DIR.mkdir(parents=True, exist_ok=True)


fertility_csv = RAW_DIR / "SP.DYN.TFRT.IN.csv"


print(fertility_csv.exists(), fertility_csv)


In [ ]:
df_raw = pd.read_csv(fertility_csv, skiprows=4)
df_raw.shape


In [ ]:
df_raw.head()


In [ ]:
# Keep only what we need: Country Name, Country Code, Years (1960–2018)
id_cols = ["Country Name", "Country Code"]
year_cols = [str(y) for y in range(1960, 2019)]  # 1960–2018 dahil

# Bazı yıllar dosyada yoksa (örneğin 2018 sonrası), sadece olanları seç
available_years = [c for c in year_cols if c in df_raw.columns]

df = df_raw[id_cols + available_years].copy()
df.head()


In [ ]:
# Convert wide -> long format
df_long = df.melt(
    id_vars=id_cols,
    var_name="Year",
    value_name="FertilityRate"
)

# Türleri düzelt
df_long["Year"] = df_long["Year"].astype(int)
df_long["FertilityRate"] = pd.to_numeric(df_long["FertilityRate"], errors="coerce")

df_long.head()


In [ ]:
df_long = df_long.dropna(subset=["FertilityRate"])

output_path = PROC_DIR / "fertility_1960_2018_long.csv"
df_long.to_csv(output_path, index=False)

print("Saved:", output_path.exists(), "| Rows:", len(df_long))
